In [ ]:
! pip install hyperopt
! pip install scikit-learn-intelex

In [1]:
%run search_model.py

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Params testing:                                        
{'bootstrap': False, 'feature_extractor': {'k': 400, 'type': 'SelectKBest'}, 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 6, 'min_samples_split': 12, 'n_estimators': 100, 'preprocessor': 'StandardScaler', 'warm_start': True}
Params testing:                                                                  
{'bootstrap': False, 'feature_extractor': {'n_components': 200, 'type': 'pca', 'whiten': False}, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 6, 'min_samples_split': 3, 'n_estimators': 100, 'preprocessor': 'Normalizer', 'warm_start': False}
Exception : Too much time, move on to next cross validate                        
Exception : Too much time, move on to next cross validate                        
Exception : Too much time, move on to next cross validate                        
Params testing:                                                                    
{'bootstrap': False, 'feature_extr

1st try : max evals : 100, max runtime : 20, cv: 3

Best estimator (0.6915) : 
feature_extractor= SelectKBest with k=400, 
n_estimators = 100, 
max_depth = None; 
min_samples_split=3, 
min_samples_leaf= 1, 
preprocessor= StandardScaler, 
warm_start = False, 
bootstrap = True 

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
X = final_train.values
X_std = StandardScaler().fit_transform(X)
pca = PCA(n_components=50,whiten=True)
pca.fit(X_std)
X_pca = pca.transform(X_std)

In [11]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

cv = cross_validate(estimator=RandomForestRegressor(n_estimators=300,
                                                    max_depth = 20, 
                                                    min_samples_split=2, 
                                                    min_samples_leaf= 2), 
                    X=X_pca, 
                    y=y_train,
                    cv=5, 
                    scoring=["neg_median_absolute_error", "neg_root_mean_squared_error"],
                    n_jobs=-1,
                    verbose=0)

print(f"Mean MAE over 10 folds = {-cv['test_neg_median_absolute_error'].mean():.4f}")
print(f"Mean RMSE over 10 folds = {-cv['test_neg_root_mean_squared_error'].mean():.4f}")

Mean MAE over 10 folds = 0.5595
Mean RMSE over 10 folds = 0.8785


In [ ]:
    @timeout_decorator.timeout(20, timeout_exception=StopIteration)
    def cross_val(model, X, y, cv):
        y_cv_predict = cross_val_predict(model, X, y, cv=cv, n_jobs=None)
        acc = mean_absolute_error(y, y_cv_predict)
        return(acc)

    n_repeats = 3
    acc = np.zeros(n_repeats)
    timeout = np.zeros(n_repeats)
    for i in range(n_repeats):
        print(i)
        try:
            acc[i]=cross_val(model, X, y, 5)

        except RuntimeError as e:
            print("err")
            if "generator raised StopIteration" in str(e):
                print("p2")
                timeout[i]=1
                print(f"Exception : Too much time, move on to next cross validate")
            else:
                raise
        
    if np.sum(timeout)==n_repeats:
        print("p5")
        mae=300
    else:
        print("p6")
        mae=acc[timeout==0].mean()